In [17]:
import sqlite3
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import joblib


In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('Education.db')  # Replace with your actual database name
cursor = conn.cursor()

In [4]:

# Load the SQL extension
%load_ext sql

# Connect to the SQLite database and specify the filename
%sql sqlite:///Education.db

# Define the SQL commands as a multiline string
sql_commands = """
CREATE TABLE Subjects (
    subject_id INTEGER PRIMARY KEY,
    subject_name TEXT NOT NULL
);

CREATE TABLE DifficultyLevels (
    difficulty_id INTEGER PRIMARY KEY,
    difficulty_name TEXT NOT NULL
);

CREATE TABLE Topics (
    topic_id INTEGER PRIMARY KEY,
    topic_name TEXT NOT NULL
);

CREATE TABLE Questions (
    question_id INTEGER PRIMARY KEY,
    question_text TEXT NOT NULL,
    subject_id INTEGER,
    difficulty_id INTEGER,
    topic_id INTEGER,
    FOREIGN KEY (subject_id) REFERENCES Subjects(subject_id),
    FOREIGN KEY (difficulty_id) REFERENCES DifficultyLevels(difficulty_id),
    FOREIGN KEY (topic_id) REFERENCES Topics(topic_id)
);

CREATE TABLE VideoFiles (
    video_id INTEGER PRIMARY KEY,
    file_path TEXT NOT NULL
);

CREATE TABLE Videos (
    video_id INTEGER PRIMARY KEY,
    video_title TEXT NOT NULL,
    subject_id INTEGER,
    teacher_id INTEGER,
    file_id INTEGER,
    FOREIGN KEY (subject_id) REFERENCES Subjects(subject_id),
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id),
    FOREIGN KEY (file_id) REFERENCES VideoFiles(video_id)
);

CREATE TABLE Teachers (
    teacher_id INTEGER PRIMARY KEY,
    teacher_name TEXT NOT NULL,
    specialty TEXT
);
"""

# Execute the SQL commands
%sql $sql_commands

# Retrieve table names
%sql SELECT name FROM sqlite_master WHERE type='table';


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///Education.db
(sqlite3.OperationalError) table Subjects already exists
[SQL: CREATE TABLE Subjects ( subject_id INTEGER PRIMARY KEY, subject_name TEXT NOT NULL );]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 * sqlite:///Education.db
Done.


name
Subjects
DifficultyLevels
Topics
Questions
VideoFiles
Videos
Teachers


In [5]:
# Insert sample data for Subjects
%sql INSERT INTO Subjects (subject_name) VALUES ('Math'), ('English'), ('Science');

# Insert sample data for DifficultyLevels
%sql INSERT INTO DifficultyLevels (difficulty_name) VALUES ('Easy'), ('Medium'), ('Hard');

# Insert sample data for Topics
%sql INSERT INTO Topics (topic_name) VALUES ('Algebra'), ('Grammar'), ('Biology');

# Insert sample data for Questions
%sql INSERT INTO Questions (question_text, subject_id, difficulty_id, topic_id) VALUES \
    ('What is 2 + 2?', 1, 1, 1), \
    ('Identify the subject in "The cat is on the mat."', 2, 1, 2), \
    ('What is the powerhouse of the cell?', 3, 2, 3);

# Insert sample data for VideoFiles
%sql INSERT INTO VideoFiles (file_path) VALUES \
    ('/videos/math_video.mp4'), \
    ('/videos/english_video.mp4'), \
    ('/videos/science_video.mp4');

# Insert sample data for Teachers
%sql INSERT INTO Teachers (teacher_name, specialty) VALUES \
    ('Mr. Smith', 'Math'), \
    ('Mrs. Johnson', 'English'), \
    ('Dr. Davis', 'Biology');

# Insert sample data for Videos
%sql INSERT INTO Videos (video_title, subject_id, teacher_id, file_id) VALUES \
    ('Math Basics', 1, 1, 1), \
    ('Grammar Rules', 2, 2, 2), \
    ('Introduction to Biology', 3, 3, 3);


 * sqlite:///Education.db
3 rows affected.
 * sqlite:///Education.db
3 rows affected.
 * sqlite:///Education.db
3 rows affected.
 * sqlite:///Education.db
3 rows affected.
 * sqlite:///Education.db
3 rows affected.
 * sqlite:///Education.db
3 rows affected.
 * sqlite:///Education.db
3 rows affected.


[]

In [6]:
%sql SELECT * FROM Subjects;

 * sqlite:///Education.db
Done.


subject_id,subject_name
1,Math
2,English
3,Science


In [7]:
%%sql
SELECT *
FROM Questions
WHERE difficulty_id = 1;


 * sqlite:///Education.db
Done.


question_id,question_text,subject_id,difficulty_id,topic_id
1,What is 2 + 2?,1,1,1
2,Identify the subject in The cat is on the mat.,2,1,2


In [8]:
import sqlite3
import random

# Connect to the SQLite database
conn = sqlite3.connect('Education.db')  # Replace with your actual database name
cursor = conn.cursor()

# Define the parameters for the diagnostic test
subject_id = 1  # Replace with the actual subject ID
difficulty_id = 1  # Replace with the actual difficulty ID
topic_id = 1  # Replace with the actual topic ID
num_questions = 5  # Replace with the desired number of questions in the diagnostic test

# Select questions based on difficulty, topic, and subject
query = """
    SELECT question_id, question_text
    FROM Questions
    WHERE subject_id = ? AND difficulty_id = ? AND topic_id = ?
    ORDER BY RANDOM()
    LIMIT ?
"""

cursor.execute(query, (subject_id, difficulty_id, topic_id, num_questions))
diagnostic_test_questions = cursor.fetchall()

# Display the selected questions
print("Diagnostic Test Questions:")
for question_id, question_text in diagnostic_test_questions:
    print(f"{question_id}. {question_text}")

# Close the database connection
conn.close()

Diagnostic Test Questions:
1. What is 2 + 2?


In [10]:
# Connect to the SQLite database
conn = sqlite3.connect('Education.db')  # Replace with your actual database name
cursor = conn.cursor()

# Create Students table
cursor.execute('''
    CREATE TABLE Students (
        student_id INTEGER PRIMARY KEY,
        performance REAL NOT NULL
    );
''')

# Insert sample data into Students table
cursor.executemany('''
    INSERT INTO Students (student_id, performance) VALUES (?, ?);
''', [(1, 80.5), (2, 92.3), (3, 75.0)])

# Create DiagnosticTests table
cursor.execute('''
    CREATE TABLE DiagnosticTests (
        test_id INTEGER PRIMARY KEY,
        student_id INTEGER,
        diagnostic_result TEXT NOT NULL,
        learning_plan TEXT,
        FOREIGN KEY (student_id) REFERENCES Students(student_id)
    );
''')

# Insert sample data into DiagnosticTests table
cursor.executemany('''
    INSERT INTO DiagnosticTests (student_id, diagnostic_result, learning_plan) VALUES (?, ?, ?);
''', [(1, 'High', 'Advanced'), (2, 'Medium', 'Intermediate'), (3, 'Low', 'Basic')])

# Commit the changes
conn.commit()

# Close the database connection
conn.close()

# Connect to the SQLite database again to fetch data into a Pandas DataFrame
conn = sqlite3.connect('Education.db')  # Replace with your actual database name
cursor = conn.cursor()

# Retrieve student performance data and diagnostic test results
query = '''
    SELECT Students.student_id, Students.performance, DiagnosticTests.diagnostic_result, DiagnosticTests.learning_plan
    FROM Students
    JOIN DiagnosticTests ON Students.student_id = DiagnosticTests.student_id;
'''

# Execute the query and fetch data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
df

,student_id,performance,diagnostic_result,learning_plan
0,1,80.5,High,Advanced
1,2,92.3,Medium,Intermediate
2,3,75.0,Low,Basic


In [19]:
# Connect to the SQLite database
conn = sqlite3.connect('Education.db')  # Replace with your actual database name
cursor = conn.cursor()

# Retrieve student performance data and diagnostic test results
query = """
    SELECT Students.student_id, Students.performance, DiagnosticTests.diagnostic_result AS diagnostic_result, DiagnosticTests.learning_plan
    FROM Students
    JOIN DiagnosticTests ON Students.student_id = DiagnosticTests.student_id;
"""

# Execute the query and fetch data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
df.head()

# Prepare features (X) and target variable (y)
X = df[['performance', 'diagnostic_result']]
y = df['learning_plan']

# Encode categorical variables if needed (e.g., diagnostic_result)
label_encoder = LabelEncoder()
X = X.copy()
X['diagnostic_result'] = label_encoder.fit_transform(X['diagnostic_result'])


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
model = DecisionTreeClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy}')

# Save the trained model for future use
joblib.dump(model, 'learning_plan_model.joblib')

Model Accuracy: 0.0


['learning_plan_model.joblib']

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('Education.db')  # Replace with your actual database name
cursor = conn.cursor()

# Define score ranges and corresponding learning plans
score_ranges = {
    (70, 100): 'Advanced',
    (50, 69): 'Intermediate',
    (0, 49): 'Basic'
}

# Retrieve student performance data
query = '''
    SELECT student_id, performance
    FROM Students;
'''
cursor.execute(query)
performance_data = cursor.fetchall()

# Determine learning plan based on score ranges
learning_plans = []
for student_id, score in performance_data:
    for score_range, plan in score_ranges.items():
        if score_range[0] <= score <= score_range[1]:
            learning_plans.append((student_id, plan))

# Create a LearningPlans table
cursor.execute('''
    CREATE TABLE LearningPlans (
        student_id INTEGER PRIMARY KEY,
        plan TEXT NOT NULL
    );
''')

# Insert learning plans into the LearningPlans table
cursor.executemany('''
    INSERT INTO LearningPlans (student_id, plan) VALUES (?, ?);
''', learning_plans)

# Commit the changes
conn.commit()

# Close the database connection
conn.close()


In [22]:
# Connect to the SQLite database
conn = sqlite3.connect('Education.db')  # Replace with your actual database name

# Read the LearningPlans table into a Pandas DataFrame
query = '''
    SELECT *
    FROM LearningPlans;
'''
df_learning_plans = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df_learning_plans)

   student_id      plan
0           1  Advanced
1           2  Advanced
2           3  Advanced
